In [ ]:
import requests
import subprocess
import pandas as pd
import datetime
%matplotlib inline

proc = subprocess.Popen(
    [
        "scp",
        "translate:/home/holidaycheck/Notebooks/Tony/Other/coinmarketcap.csv",
        "."
    ],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)
proc.wait()
print("RESULT: %s" % repr(proc.stderr.readline()))

In [ ]:
changelly_currencies = [
    "Bitcoin", "Litecoin", "Bytecoin",
    "Monero", "DigitalNote", "QuazarCoin",
    "Dashcoin", "Fantomcoin", "Dogecoin",
    "Ethereum", "NuBits", "Dash",
    "Ripple", "Next", "Aeon",
    "Radium", "MaidSafeCoin", "Factom",
    "Synereo", "Lisk", "Ethereum Classic",
    "Expanse", "NEM", "Steem",
    "LBRY Credits", "Stratis", "Ardor",
    "Gulden", "NAV Coin", "GameCredits",
    "PotCoin", "Tether", "Golem",
    "Swarm City", "Waves", "Melon",
    "Golos", "DIGIX", "SNGLS",
    "Chronobank", "Xaurum", "PIVX",
    "Trustcoin", "Edgeless", "iExec RLC",
    "Wings", "Byteball", "Darcrus",
    "Gnosis", "Guppy", "Decred",
    "Syscoin", "Stellar", "Basic Attention Token",
    "Lunyr", "Aragon", "Bancor",
    "Status", "Civic", "TenX",
    "Qtum", "EOS", "Bitcoin Cash",
    "Neo", "FirstBlood", "OmiseGo",
    "AdEx", "Monaco", "0x Protocol",
    "Metal"
]

In [ ]:
coinomi_currencies = [
    "Bitcoin", "Abncoin", "Asiacoin", "Auroracoin", "Bata", "Befacoin", "Bitcoin Plus",
    "Blackcoin", "Britcoin", "Canada eCoin", "Cannacoin", "Clams", "Clubcoin",
    "Compcoin", "Dash", "Defcoin", "Denarius", "Digibyte", "Digitalcoin", "Dogecoin",
    "Ecoin", "EDRCoin", "EGulden", "Einsteinium", "Ethereum", "Ethereum Classic", "Expanse",
    "Feathercoin", "Flashcoin", "Game Credits", "GCRcoin", "Gridcoin", "Gulden", "Hempcoin",
    "Insanecoin", "Internet of People", "IXCoin", "Jumbucks", "Landcoin", "LBRY Credits", "Linx",
    "Litecoin", "Monacoin", "Myriadcoin", "Namecoin", "Navcoin", "Neoscoin", "Neurocoin",
    "Novacoin", "NuBits", "NuShares", "OKCash", "ParkByte", "Peercoin", "Pesobit", "Pinkcoin",
    "PIVX", "POSW", "Potcoin", "Putincoin", "Reddcoin", "Richcoin", "Rubycoin", "ShadowCash",
    "SmileyCoin", "Solarcoin", "Stratis", "Syscoin", "Ultimate Secure Cash", "Unobtanium",
    "Verge", "Vertcoin", "Vpncoin", "ZCash", "ZCoin"
]

In [ ]:
def calc_summary(_df):
    results = []
    
    # process rows
    for ix, grp in _df.groupby("name"):
        rank_diff = grp["rank"].diff()
        result = {
            "name": ix,
            "minRank": grp["rank"].min(),
            "maxRank": grp["rank"].max(),
            "negRank": sum(rank_diff > 0),
            "posRank": sum(rank_diff < 0),
            "last100RankChanges": rank_diff[rank_diff != 0].tail(100).sum(),
            "currentRank": grp["rank"].tail(1).item(),
            "minUSD": grp.price_usd.min(),
            "maxUSD": grp.price_usd.max(),
            "curUSD": grp.price_usd.tail(1).item()
        }
        results.append(result)
    
    # create new data frame
    _tmp = pd.DataFrame(results, columns=[
        "name", "currentRank", "minRank", "maxRank",
        "last100RankChanges", "negRank", "posRank", "minUSD", "maxUSD", "curUSD"
    ])
    _tmp["perUSD"] = (_tmp.maxUSD - _tmp.curUSD) / _tmp.curUSD
    
    return _tmp

In [ ]:
df = pd.read_csv("coinmarketcap.csv")
df["date"] = df.last_updated.apply(lambda x: datetime.datetime.fromtimestamp(x))
df.date.min(), df.date.max(), df.shape

In [ ]:
tmp = calc_summary(df[df.name.isin(coinomi_currencies) & (df.date > '2017-11-30')])
tmp.sort_values(by="last100RankChanges").query("posRank < negRank").round(decimals=5)

In [ ]:
tmp = calc_summary(df[
    df.name.isin(set(coinomi_currencies) & set(changelly_currencies)) & (df.date > '2017-11-30')
])
tmp.sort_values(by="last100RankChanges").query("posRank > negRank").round(decimals=5)

In [ ]:
tmp = calc_summary(df[
    (df.date > '2017-11-30')
])
tmp.sort_values(by="last100RankChanges").query("posRank-5 > negRank").round(decimals=5)

In [ ]:
tmp = calc_summary(df[df.name.isin(
    set(coinomi_currencies) & set(changelly_currencies)
)])
tmp.sort_values(by="last100RankChanges").query("posRank > negRank").round(decimals=5)

In [ ]:
tmp = calc_summary(df[df.name.isin(coinomi_currencies)])
tmp.sort_values(by="last100RankChanges").query("posRank > negRank").round(decimals=5)

In [ ]:
tmp = calc_summary(df[df.name.isin(
    set(changelly_currencies) & set(coinomi_currencies)
) & (df.date > '2017-11-01')])
tmp.sort_values(by="last100RankChanges").round(decimals=5)

In [ ]:
tmp = calc_summary(df[df.name.isin(
    set(changelly_currencies) & set(coinomi_currencies)
) & (df.date > '2017-10-01')])
tmp.sort_values(by="last100RankChanges").query("posRank > negRank").round(decimals=5)

In [ ]:
tmp = calc_summary(df[df.name.isin(changelly_currencies)])
tmp.sort_values(by="last100RankChanges").query("posRank < negRank").round(decimals=5)

In [ ]:
tmp = calc_summary(
    df[df.name.isin(changelly_currencies) & (df.date > '2017-09-11')]
)
tmp.sort_values(by="last100RankChanges").query("posRank > negRank").round(decimals=5)

In [ ]:
tmp = calc_summary(
    df[df.name.isin(changelly_currencies) & (df.date > '2017-09-12')]
)
tmp.sort_values(by="perUSD", ascending=False).round(decimals=5)

In [ ]:
tmp = calc_summary(df[df.name.isin(coinomi_currencies) & ~df.name.isin(changelly_currencies)])
tmp.sort_values(by="last100RankChanges").query("posRank > negRank").round(decimals=5)

In [ ]:
tmp = calc_summary(df[~df.name.isin(changelly_currencies)])
tmp.sort_values(by="last100RankChanges").query("posRank > negRank").round(decimals=5)

In [ ]:
df.query("name == 'IOTA' & date > '2017-10-30'").price_usd.plot(figsize=(12,4))

In [ ]:
df.query("name == 'MonaCoin'").pr

In [ ]:
df.head()